In [1]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import threading
import time

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Button("Start Long Process", id="start-btn", className="mr-2"),
            dbc.Spinner(dcc.Interval(id="interval-component", interval=1*1000, max_intervals=0)),
        ])
    ]),
    html.Div(id="output"),
])

@app.callback(
    Output("output", "children"),
    Output("interval-component", "max_intervals"),
    Input("start-btn", "n_clicks"),
    State("interval-component", "n_intervals")
)
def long_running_process(n, n_intervals):
    if n:
        if n_intervals == 0:
            def target():
                time.sleep(10)  # This emulates a long process
                app.server.store["finished"] = True

            thread = threading.Thread(target=target)
            thread.start()
            return "Started...", 10  # Allow the interval to tick 10 times (10 seconds)
        elif app.server.store.get("finished"):
            return "Process finished!", 0  # Reset the interval
    return "", 0  # Default case

if __name__ == '__main__':
    app.server.store = {}
    app.run_server(debug=True)

"""
In this example, we use app.server.store to maintain state across callbacks without using global variables. It acts like a shared memory across threads.
This is a simple example, and in real-world applications, you might need more sophisticated error handling and mechanisms to keep track of running threads.
If your long-running tasks are even more time-consuming or need to be offloaded from your main application server, you might want to consider using solutions like Celery with Redis or RabbitMQ.
"""

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


C:\Users\zeesh\AppData\Local\Temp\ipykernel_14240\3007778817.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\zeesh\AppData\Local\Temp\ipykernel_14240\3007778817.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


SystemExit: 1

D:\workbook\Anaconda3\envs\test\Lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
"""
Here are steps to refactor your application:

Thread-Safe Shared Data:
Use a threading.Lock to protect shared data. This ensures that only one thread can access the data at any given time.

Using Flask's g Object:
Flask’s g object can be used to store data that is specific to an individual request. You can use this to store session-specific data like your image path.

Here's a general structure to demonstrate the approach:
------------------------------------------------------

Notes:

This example uses Flask's g object to store the path to the image specific to each session/request.
We use a lock to ensure that the shared data (image path) is accessed in a thread-safe manner.
When the dropdown value changes, the callback starts a new thread that simulates generating and saving an image based on the selected dropdown value. This thread-safe method updates the g.image_path.
Adjust this example to fit the specifics of your application. If you're generating and storing real images, you'd replace the mock image generation (image_path = f"{value}_image.png") with your actual logic.

Lastly, remember to handle potential errors and exceptions, especially in the threaded function, to ensure the stability of your application.


"""
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import threading
from flask import g

app = dash.Dash(__name__)
app.server.secret_key = 'secret'

app.layout = html.Div([
    dcc.Dropdown(
        id='dropdown',
        options=[{'label': i, 'value': i} for i in ['Option 1', 'Option 2', 'Option 3']],
        value='Option 1'
    ),
    html.Div(id='waterfall-output'),
    # ... rest of your layout ...
])

lock = threading.Lock()

def generate_waterfall_image(value):
    # Mocking the generation of an image path based on dropdown value
    image_path = f"{value}_image.png"
    with lock:
        g.image_path = image_path
    # Here, you can implement the logic to save the image on the server

@app.callback(
    Output('waterfall-output', 'children'),
    Input('dropdown', 'value')
)
def update_output(value):
    # Start the thread to save the image
    threading.Thread(target=generate_waterfall_image, args=(value,)).start()
    
    # For this demonstration, we're just returning the dropdown value to the output
    # In a real scenario, you would use the generated image path (stored in g.image_path)
    # to display the waterfall image on your Dash page.
    return f'Selected: {value}'

if __name__ == '__main__':
    app.run_server()



Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


Exception in thread Thread-12 (generate_waterfall_image):
Traceback (most recent call last):
  File "D:\workbook\Anaconda3\envs\test\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\workbook\Anaconda3\envs\test\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\zeesh\AppData\Local\Temp\ipykernel_14240\1165278010.py", line 50, in generate_waterfall_image
  File "D:\workbook\Anaconda3\envs\test\Lib\site-packages\werkzeug\local.py", line 311, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\workbook\Anaconda3\envs\test\Lib\site-packages\werkzeug\local.py", line 508, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See t

In [ ]:
In a Dash application, the underlying Flask server (app.server) provides the core functionality to serve HTTP requests. If you're trying to use a shared dictionary (app.server.store = {}) across multiple threads, you need to be careful because standard Python dictionaries are not thread-safe.

Using a standard dictionary with multiple threads can lead to unexpected behaviors, like missing data or corrupted state. To handle shared data in a multi-threaded environment, you have a few options:

Use a thread-safe collection: The Python collections module provides a thread-safe dictionary called collections.defaultdict.

Use thread synchronization primitives: The threading module provides synchronization primitives like Locks, Semaphores, etc. You can use a Lock to ensure that only one thread modifies the dictionary at a time.

In [3]:
import os
from uuid import uuid4
from threading import Lock
from dash import Dash, dcc, html, Input, Output
from flask import session

# Create a base directory for all user directories
BASE_DIR = './user_dirs'

# Ensure the base directory exists
if not os.path.exists(BASE_DIR):
    os.makedirs(BASE_DIR)

app = Dash(__name__)
app.server.secret_key = 'your_secret_key'  # Needed for Flask session
directory_creation_lock = Lock()

app.layout = html.Div([
    html.Button('Create My Directory', id='create-btn'),
    html.Div(id='dir-output')
])

@app.callback(
    Output('dir-output', 'children'),
    Input('create-btn', 'n_clicks')
)
def create_directory(n):
    if n is None:
        # Button not clicked yet
        return "Click the button to create your directory."
    
    if 'directory_path' not in session:
        with directory_creation_lock:
            unique_dir = str(uuid4())
            dir_path = os.path.join(BASE_DIR, unique_dir)
            os.makedirs(dir_path, exist_ok=True)
            session['directory_path'] = dir_path
            
    return f"Your directory path is: {session['directory_path']}"

if __name__ == '__main__':
    app.run_server( threaded=True)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


In [5]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash_bootstrap_components import Tabs, Tab
import threading
import time

external_stylesheets = ['https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    Tabs(id='tabs', active_tab='tab-1', children=[
        Tab(label='Tab 1', tab_id='tab-1'),
        Tab(label='Tab 2', tab_id='tab-2'),
    ]),
    html.Div(id='tabs-content'),
    html.Div(id='hidden-div', style={'display': 'none'}),
])


@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'active_tab')
)
def render_content(tab):
    # Depending on the tab, you might want to display different content.
    if tab == 'tab-1':
        return html.Div([
            html.H3('Tab 1 Content')
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab 2 Content')
        ])


# Use a global variable to keep track of threads
current_thread = None
thread_stop_event = threading.Event()

@app.callback(
    Output('hidden-div', 'children'),
    Input('tabs', 'active_tab')
)
def manage_threads(active_tab):
    global current_thread
    global thread_stop_event

    # If there's an existing thread, stop it
    if current_thread is not None:
        thread_stop_event.set()  # Signal the thread to stop
        current_thread.join()  # Wait for the thread to finish

    # Reset the event for the new thread
    thread_stop_event.clear()

    if active_tab == 'tab-1':
        current_thread = threading.Thread(target=tab1_task, args=(thread_stop_event,))
    elif active_tab == 'tab-2':
        current_thread = threading.Thread(target=tab2_task, args=(thread_stop_event,))

    current_thread.start()
    return ''  # The output doesn't matter as it's a hidden div


def tab1_task(stop_event):
    while not stop_event.is_set():
        print("Tab 1 thread running...")
        time.sleep(10)


def tab2_task(stop_event):
    while not stop_event.is_set():
        print("Tab 2 thread running...")
        time.sleep(10)


if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 2 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 2 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...Tab 1 thread running...
Tab 2 thread running...

Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread running...
Tab 1 thread runni